## Gaussian Distribution and Linear Regression

* $ Gaussian \ Distribution : \frac{1}{\sqrt{2\pi\sigma^2}}* exp[-\frac{(x - \mu)^2}{2\sigma^2}] $


* In Lin Reg, the observations are assumed to arise from noisy observations -> $\epsilon$ ~ $N(0, \sigma^2)$


* Likelihood of seeing y for given x:

 $ P(y|x) = \frac{1}{\sqrt{2\pi\sigma^2}}* exp[-\frac{(y - w^Tx - b)^2}{2\sigma^2}] $
 
 
* By principle of max likelihood, best value of w, b for entire dataset which maximize the likelihood:

    $ P(y|X) = \prod p(y|x) $
 
 
* The abv eq has prod of exp -> diff to work with -> take log and minimize with - sign

     $ -log(P(y|X)) = \sum \frac{1}{2} log (2\pi\sigma^2) + \frac{1}{2\sigma^2} (y - w^Tx - b)^2 $
 
 
* The second term is MSE and need to be minimized
 

## Scratch Implementation

In [1]:
%matplotlib inline
from d2l import tensorflow as d2l
import tensorflow as tf
import random

In [2]:
def synthetic_data(w, b, num_examples):  #@save
    """Generate y = Xw + b + noise."""
    X = tf.zeros((num_examples, w.shape[0]))
    X += tf.random.normal(shape=X.shape)
    y = tf.matmul(X, tf.reshape(w, (-1, 1))) + b
    y += tf.random.normal(shape=y.shape, stddev=0.01)
    y = tf.reshape(y, (-1, 1))
    return X, y

true_w = tf.constant([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [3]:
print('features:', features[0],'\nlabel:', labels[0])

features: tf.Tensor([-0.9137856   0.81359535], shape=(2,), dtype=float32) 
label: tf.Tensor([-0.3917893], shape=(1,), dtype=float32)


In [4]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        j = tf.constant(indices[i: min(i + batch_size, num_examples)])
        yield tf.gather(features, j), tf.gather(labels, j)

In [5]:
w = tf.Variable(tf.random.normal(shape = (2, 1), mean = 0, stddev = 0.1),
               trainable = True)
b = tf.Variable(tf.zeros(1), trainable = True)

In [6]:
def linreg(X, w, b):  #@save
    """The linear regression model."""
    return tf.matmul(X, w) + b

def squared_loss(y_hat, y): #@save
    return (y_hat - tf.reshape(y, y_hat.shape))**2/2

def sgd(params, grads, lr, batch_size):  #@save
    """Minibatch stochastic gradient descent."""
    for param, grad in zip(params, grads):
        param.assign_sub(lr*grad/batch_size)

In [7]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss
batch_size = 10

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        with tf.GradientTape() as g:
            # Do frwd prop, Compute loss
            l = loss(net(X, w, b), y) 
        
        # Compute gradient
        dw, db = g.gradient(l, [w, b])
        
        # Update w, b
        sgd([w, b], [dw, db], lr, batch_size)
    
    # Compute loss per epoch
    train_l = loss(net(features, w, b), labels)
    print(f'epoch {epoch + 1}, loss {float(tf.reduce_mean(train_l)):f}')

epoch 1, loss 0.038664
epoch 2, loss 0.000144
epoch 3, loss 0.000051


In [8]:
print(f'error in estimating w: {true_w - tf.reshape(w, true_w.shape)}')
print(f'error in estimating b: {true_b - b}')

error in estimating w: [ 0.00105822 -0.00057888]
error in estimating b: [-6.246567e-05]


## Concise Implementation

In [1]:
from d2l import tensorflow as d2l
import numpy as np
import tensorflow as tf

true_w = tf.constant([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [2]:
def load_array(data_arrays, batch_size, is_train = False): #@save
    """ Construct Tensorflow data iterator
    Rather using own iterator, we call upon existing API
    is_train : to shuffle data """
    dataset = tf.data.Dataset.from_tensor_slices(data_arrays)
    if is_train:
        dataset = dataset.shuffle(buffer_size = 1000)
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [3]:
next(iter(data_iter))

(<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
 array([[-0.7856717 ,  0.36311144],
        [-1.3061655 ,  0.86732227],
        [ 0.8133625 ,  0.3144101 ],
        [ 0.79285336, -1.4506221 ],
        [-0.76973766,  0.00992054],
        [ 0.62765217,  0.41903365],
        [ 1.4488863 , -1.2609822 ],
        [-1.4496723 , -0.96479976],
        [-0.5007983 ,  1.095517  ],
        [-0.8820045 ,  0.8823762 ]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[ 1.3964784 ],
        [-1.358684  ],
        [ 4.773747  ],
        [10.715026  ],
        [ 2.6173177 ],
        [ 4.0368996 ],
        [11.38818   ],
        [ 4.58385   ],
        [-0.53008974],
        [-0.55761766]], dtype=float32)>)

In [4]:
# intialize each param in weights by std normal
initializer = tf.initializers.RandomNormal(stddev = 0.01)
net = tf.keras.Sequential()
net.add(tf.keras.layers.Dense(1, kernel_initializer = initializer))

In [5]:
trainer = tf.keras.optimizers.SGD(learning_rate=0.03)
loss = tf.keras.losses.MeanSquaredError()


In [6]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        with tf.GradientTape() as tape:
            l = loss(net(X, training =  True), y)
        
        grads = tape.gradient(l, net.trainable_variables)
        
        trainer.apply_gradients(zip(grads, net.trainable_variables))
        
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000295
epoch 2, loss 0.000100
epoch 3, loss 0.000100
